In [1]:
import time
import pickle
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('../data/finalDataSet2018to2021.csv')

In [3]:
X = df.loc[:, df.columns != 'hitTF'].select_dtypes(['int', 'float']).values
y = df.hitTF.values

In [4]:
def LogRegModel(X, y, size=None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=size + 0.05)
    
    X_scaler = MinMaxScaler().fit(X_train)

    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    X_val_scaled = X_scaler.transform(X_val)
    
    LogReg = LogisticRegression()
    
    t = time.time()
    LogReg.fit(X_train_scaled, y_train)
    elapsed_time = time.time() - t
    print(elapsed_time)
    
    print(f"Training Data Score: {LogReg.score(X_train_scaled, y_train)}")
    print(f"Testing Data Score: {LogReg.score(X_test_scaled, y_test)}", '\n')
    
    predictions = LogReg.predict(X_val_scaled)
    
    print(classification_report(y_val, predictions, target_names=['not hit','hit']), '\n')
    
    pickle.dump(LogReg, open('../deployment/models/LogReg.sav', 'wb'))
    pickle.dump(X_scaler, open('../deployment/models/scaler.gz', 'wb'))
    return LogReg.score(X_train_scaled, y_train), LogReg.score(X_test_scaled, y_test)

In [5]:
LogRegModel(X, y, 0.2)

0.3652839660644531
Training Data Score: 0.9847560975609756
Testing Data Score: 0.9827411167512691 

              precision    recall  f1-score   support

     not hit       0.98      1.00      0.99       850
         hit       1.00      0.86      0.92       135

    accuracy                           0.98       985
   macro avg       0.99      0.93      0.96       985
weighted avg       0.98      0.98      0.98       985
 



(0.9847560975609756, 0.9827411167512691)